# Lab 2.2: Minimum Temperature by Location

## Tổng quan bài tập
**Đề bài**: Dựa vào tập dữ liệu, hãy tìm nhiệt độ `TMIN` thấp nhất theo từng khu vực. Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán trên.

## Tài nguyên tham khảo

Bạn có thể tải tập Dataset tại [link sau](https://drive.google.com/file/d/1fLLX1e4Wd_Ks-0u9iRfJipVTqsVm0Kzp/view?usp=sharing). Sau đó đưa lên Google Drive và kết nối với Colab là có thể sử dụng được. Tập dữ liệu là file .csv gồm 4 cột theo thứ tự sau:
- `StationID`: Mã thành phố.
- `Date`: Thời gian.
- `Type`: Loại nhiệt độ (`TMAX` hoặc `TMIN`) tương ứng với nhiệt độ cao nhát hoặc thấp nhất trong ngày đó.
- `Temperature`: Giá trị nhiệt độ.

Ngoài ra, bạn có thể tham khảo các video sau trong trường hợp chưa hiểu cách làm bài Lab:
- [Minimum Temperature by Location](https://fpt-software.udemy.com/course/taming-big-data-with-apache-spark-hands-on/learn/lecture/3719528#overview)


# Cài đặt Spark trên Google Colab

Để có thể sử dụng Spark trên môi trường Google Colab thì bạn sẽ cần cài đặt một số thành phần sau:
- Java 8
- Spark Binary
- findspark

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [632 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,265 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,494 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:13 http://

Sau đó, bạn sẽ cần khai báo cho hệ thống các đường dẫn cho các thành phần vừa cài.

In [2]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()

# Kết nối với Google Drive

Để lấy dữ liệu từ các Dataset, bạn sẽ phải lưu file dữ liệu lên Google Drive. Sau đó kết nối Colab đến Google Drive của bạn và lấy được các file dữ liệu.

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# Minimum Temperature by Location

Bạn sẽ cần khởi tạo 1 SparkSesson để có thể bắt đầu Spark.

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('lab2.2')

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

Đọc dữ liệu từ Dataset và lưu vào RDD.

In [5]:
DATASET_PATH = '/content/gdrive/My Drive/DEP303/1800.csv'
lines = sc.textFile(DATASET_PATH)

Sau đó sẽ chuyển đổi dữ liệu thành các cặp Key-Vaule tương ứng

In [6]:
def parseLine(line):
    cells = line.split(',')
    stationID = cells[0]
    entryType = cells[2]
    # Chuyển đổi nhiệt độ về độ F
    temperature = float(cells[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

parsedLines = lines.map(parseLine)

In [7]:
# Lọc các giá trị có Type là TMIN
minTemps = parsedLines.filter(lambda x: x[1]=='TMIN')

# Tìm giá trị nhiệt độ nhỏ nhất theo các thành phố.
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: min(x, y))

results = minTemps.collect();
for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554	5.36F
EZE00100082	7.70F
